In [4]:
from pathlib import Path

import pandas as pd

from utils import check_date_sequence_integrity

from osgeo import gdal
import numpy as np
import xarray as xr

In [5]:
# 首先应该检查文件序列日期上的完整性

flt_dir = Path(r'D:\PROJECTS\hnsw_soil_moisture_surface\auto_anu\pkg_run3\data\output')

filename_list = [f.name for f in flt_dir.glob('*.flt')]
check_date_sequence_integrity(filename_list)

文件日期序列是连续的.
起始日期: 2018-01-01
结束日期: 2020-12-31


In [3]:
file_list = list(flt_dir.glob('*.flt'))

first_flt = file_list[0]
dst1 = gdal.Open(str(first_flt))
data1 = dst1.GetRasterBand(1).ReadAsArray()
geotransform = dst1.GetGeoTransform()

lon = np.arange(geotransform[0], geotransform[0] + geotransform[1] * data1.shape[1], geotransform[1])
lat = np.arange(geotransform[3], geotransform[3] + geotransform[5] * data1.shape[0], geotransform[5])

In [4]:
data = np.zeros((len(file_list), data1.shape[0], data1.shape[1]), dtype=np.float32)

# 循环读入文件放入data数组
for i, f in enumerate(file_list):
    dst = gdal.Open(str(f))
    data[i] = dst.GetRasterBand(1).ReadAsArray()
    dst = None
    
data

array([[[25.116102, 24.858206, 26.32911 , ..., 22.48185 , 22.02262 ,
         21.84563 ],
        [24.158068, 23.8872  , 24.584808, ..., 21.999859, 21.895569,
         22.130507],
        [23.611773, 25.066671, 23.971025, ..., 22.617037, 21.666746,
         21.482912],
        ...,
        [18.849394, 18.893135, 18.8382  , ..., 19.950043, 19.858337,
         19.178257],
        [18.728128, 18.766193, 18.69763 , ..., 19.058037, 18.70221 ,
         18.458189],
        [18.51089 , 18.47648 , 18.52458 , ..., 18.680275, 18.21392 ,
         18.590597]],

       [[25.174957, 24.911297, 26.215591, ..., 22.26037 , 21.819044,
         21.632011],
        [24.296013, 24.020254, 24.62464 , ..., 21.818996, 21.69718 ,
         21.88089 ],
        [23.790958, 25.081188, 24.057251, ..., 22.366758, 21.48318 ,
         21.289865],
        ...,
        [19.35315 , 19.36473 , 19.28515 , ..., 19.785265, 19.679054,
         19.041752],
        [19.23177 , 19.238337, 19.146606, ..., 18.971312, 18.626717,
   

In [5]:
# 生成时间序列
date_range = pd.date_range(start='2018-01-01', periods=len(file_list), freq='D')

In [6]:
# 输出到NC文件
ds = xr.Dataset(
    {
        'soil_moisture': (['time', 'latitude', 'longitude'], data)
    },
    coords={
        'time': date_range,
        'latitude': lat,
        'longitude': lon
    }
)

ds.attrs['title'] = 'Soil Moisture Content'
ds.attrs['description'] = 'This is a Soil Moisture Surface'

ds['soil_moisture'].attrs['units'] = '%'
ds['soil_moisture'].attrs['long_name'] = 'Soil Moisture Content'

ds

<xarray.Dataset>
Dimensions:        (time: 1096, latitude: 24, longitude: 24)
Coordinates:
  * time           (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2020-12-31
  * latitude       (latitude) float64 29.88 29.62 29.38 ... 24.62 24.38 24.12
  * longitude      (longitude) float64 109.1 109.4 109.6 ... 114.4 114.6 114.9
Data variables:
    soil_moisture  (time, latitude, longitude) float32 25.12 24.86 ... 18.08
Attributes:
    title:        Soil Moisture Content
    description:  This is a Soil Moisture Surface

In [7]:
ds.to_netcdf(r'D:\PROJECTS\hnsw_soil_moisture_surface\auto_anu\pkg_run2\soil_moisture3.nc')